# Import Google Colab utilities


In [ ]:
from google.colab import files      # Import the files module from Google Colab to handle file uploads
uploaded = files.upload()           # Opens a file upload dialog and stores uploaded files in a dictionary


#Import required libraries

In [ ]:
import pandas as pd                 # For handling and analyzing tabular data (CSV, DataFrames, etc.)
import numpy as np                  # For numerical operations, arrays, and mathematical computations
from sklearn.model_selection import train_test_split    # For splitting dataset into training and testing sets
from sklearn.preprocessing import StandardScaler        # For normalizing features (scaling inputs)
import tensorflow as tf             # TensorFlow for building ML/DL models
from tensorflow import keras        # Keras high-level API for TensorFlow
from tensorflow.keras import layers # To build neural network layers easily


#1. Load Data

In [ ]:
df = pd.read_csv("openclock2.csv")       # Load dataset from CSV file into a Pandas DataFrame
df.columns = df.columns.str.strip()      # Remove any leading/trailing whitespace from column names

print("Original dataset shape:", df.shape)  # Print number of rows and columns in the dataset
print(df.head())                           # Print first 5 rows to inspect dataset



# 2. Data Augmentation: mirror left/right and invert steering


In [ ]:
df_mirrored = df.copy()                                  # Make a copy of the dataset for augmentation
df_mirrored[["left", "right"]] = df_mirrored[["right", "left"]].values
# Swap "left" and "right" sensor values to simulate mirrored driving

df_mirrored["steering"] = -df_mirrored["steering"]       # Invert steering values for mirrored data

# Concatenate original dataset with the mirrored dataset
df = pd.concat([df, df_mirrored], ignore_index=True)     # Merge and reset index
print("Augmented dataset shape:", df.shape)

# 3. Features & labels

In [ ]:
X = df[["left", "right", "front"]].values    # Select sensor columns as features (inputs)
y = df["steering"].values                    # Select steering column as target (output)

# Scale steering labels into range [-1, 1]
y = y / 22.0    # Assuming max steering value is ~22, dividing normalizes steering outputs

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
# 80% training, 20% testing, random_state=42 ensures reproducibility, shuffle ensures randomness

# Normalize input features (left, right, front) so they have mean=0 and std=1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)   # Fit scaler on training set and transform it
X_test = scaler.transform(X_test)         # Transform test set using same scaler (no data leakage)


# 4. Build Model (Huber loss + zero bias)

In [ ]:
model = keras.Sequential([

    layers.Input(shape=(X_train.shape[1],)),       # Input layer, shape = number of features (3: left, right, front)

    layers.Dense(32, activation="relu"),           # First hidden layer with 32 neurons and ReLU activation
    layers.Dense(32, activation="relu"),           # Second hidden layer with 32 neurons and ReLU activation
    layers.Dense(16, activation="relu"),           # Third hidden layer with 16 neurons and ReLU activation

    layers.Dense(
        1,                                         # Output layer: single value (steering)
        activation="linear",                       # Linear activation because we want continuous output
        kernel_initializer="glorot_uniform",       # Xavier initialization for weights
        bias_initializer="zeros"                   # Start biases at 0
    )
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adam optimizer with learning rate 0.001
    loss=tf.keras.losses.Huber(),                            # Huber loss (robust to outliers)
    metrics=["mae"]                                          # Track Mean Absolute Error during training
)

# 5. Train

In [ ]:
history = model.fit(
    X_train, y_train,                     # Training data
    validation_data=(X_test, y_test),     # Validation data (to monitor overfitting)
    epochs=500,                           # Train for 500 epochs
    batch_size=32,                        # Use mini-batches of 32 samples
    verbose=1                             # Show training progress
)

# 6. Save model

In [ ]:
model.save("steering_model.h5")     # Save trained Keras model in HDF5 format
print("✅ Keras model (with mirrored negatives) saved as steering_model.h5")


# 7. Convert to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)    # Create a TFLite converter from trained Keras model
converter._experimental_lower_tensor_list_ops = True           # Enable experimental optimizations
converter.experimental_enable_resource_variables = False       # Disable resource variables (for compatibility)

tflite_model = converter.convert()    # Convert Keras model into TFLite format

# Save the TFLite model
with open("steering_model_openclock.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved as steering_model.tflite")

After the conversion to tflite model, we need to convert it into kmodel for running it in CanMV K230